In [27]:
from db import call_ignition
from queries import stock_all, proc, update_processing, post_carts 
from release_gy3 import get_stock, check_type, get_default_item_number, generate_sql_string
import pandas as pd
import numpy as np
from datetime import timedelta, date, datetime
import re
import os
stock_all()
# set_kanban(False)
update_processing()
# await grinding_exh()

In [28]:
test = proc.HKC10[:4]
grouped = test.groupby('item_description')
missed_df = grouped.first()
missed_df['order_quantity'] = grouped["order_quantity"].sum()
missed_df['order_number'] = grouped["order_number"].apply(np.array)
def generate_sql_string(row, item_number):
    cart_id = "R-" + row.order_number[0]    
    quantity = str(int(row.order_quantity))
    mos = f"ARRAY{list(row.order_number)}"
    return f"('{cart_id}', 'Warehouse', NOW(), true, false, true, null, '{cart_id}', 'GY3', '{item_number}',  {quantity}, 'RELEASED', {mos})"
for row in missed_df.itertuples():
    print(generate_sql_string(row, 1234))


('R-M146113', 'Warehouse', NOW(), true, false, true, null, 'R-M146113', 'GY3', '1234',  16, 'RELEASED', ARRAY['M146113', 'M146114'])
('R-M146107', 'Warehouse', NOW(), true, false, true, null, 'R-M146107', 'GY3', '1234',  16, 'RELEASED', ARRAY['M146107', 'M146108'])


In [30]:
def post_missing_mo_to_batches():
    sql = """
        SELECT order_id FROM ignition.production_schedule.batch_orders    
    """
    batch_mos = pd.DataFrame(call_ignition(sql))["order_id"].to_list()
    df = proc.HKC10
    missed_df = df[~df["order_number"].isin(batch_mos) & (df["facility_id"] < "F0010")]
    if len(missed_df):
        grouped = missed_df.groupby("item_description")
        missed_df_grouped = grouped.first()
        missed_df_grouped["order_quantity"] = grouped["order_quantity"].sum()
        missed_df_grouped["order_number"] = grouped["order_number"].apply(np.array)
        source_map_df = pd.DataFrame(
            columns=["rail_type", "type", "item_number"]
        ).set_index(["rail_type", "type"])
        stock_df, _ = get_stock()
        total_item = []
        missed_df_grouped = missed_df_grouped.reset_index()
        for row in missed_df_grouped.itertuples(): 
            rail_type = row.product_family + row.product_model
            print(rail_type)
            type = check_type(row.item_description, "stock")
            item_number = get_default_item_number(
                stock_df, source_map_df, rail_type, type
            )
            total_item.append(generate_sql_string(row, item_number))
        print(",".join(total_item))
        # print(missed_df_grouped.order_number.to_list())
        # return post_carts(",".join(total_item), from_back_end=True)
    # return 0

post_missing_mo_to_batches()


SR35
('R-M232161', 'Warehouse', NOW(), true, false, true, null, 'R-M232161', 'GY3', '800435200B25200',  1, 'RELEASED', ARRAY['M232161'])
